# Basic Spatially-Extended Stochastic Lotka-Volterra Model

In this first example, we want to focus only on a simple non-spatial system. In essence, this is equivalent to a lattice with only a single, isolated lattice site.

We need to install the python packages needed to run the simulations:

In [ ]:
%pip install git+https://github.com/ulido/pypop matplotlib pandas ipywidgets tqdm

To start, we need to load the `pypop` library. We only load what we need so not to clutter our environment.

In [ ]:
from pypop import World, Species, BirthReaction, DeathReaction, PredationBirthReaction, Hop

Next, we set up the world. This involves setting up the species (the predator A and the prey B) and the reactions (B can give birth, A can die and predate upon B). The following are the parameters of the simulation:
* `size`: This is the size of the 2D lattice and is specified as a tuple of two integers (the width and the height) like so `(256, 256)`. It is a good idea to start small since larger lattices take more memory and more computational time to simulate.
* `sigma`: This is the prey (species "B") birth rate. It regulates how quickly prey will reproduce.
* `mu`: This is the predator (species "A") death rate. It regulates how quickly predators are removed from the system.
* `lam`: This is the predation rate. It regulates how efficiently predators (species "A") consume prey (species "B") and reproduce at the same time.
* `hA` and `hB`: This is the hop rate, which in typically is left at a value of `1` since this will only rescale time. It could be varied for each species separately, such that prey e.g. more static than predators or vice-versa.
* `rhoA` and `rhoB`: This is the initial density of the predators (species "A") and prey (species "B"). It specifies how many occupants of each species will be present on each lattice site on average.
* `T`: Specifies the end time of the simulation, i.e. how many Monte Carlo steps will be performed.

In [ ]:
# The simulation parameters
size = (100, 100)
sigma: float = 0.1
mu: float = 0.1
lam: float = 0.1
hA: float = 1.0
hB: float = 1.0
rhoA: float = 0.01
rhoB: float = 1.0
T: int = 1000

# Predator and prey species
A = Species("A")
B = Species("B")

# The reactions
reactions = {
    A: [
        PredationBirthReaction(A, B, lam),
        DeathReaction(A, mu),
    ],
    B: [
        BirthReaction(B, sigma),
    ]
}

# Initialization of the world
world = World(
    size=size,
    initial_densities={B: rhoB, A: rhoA},
    hops={A: Hop(A, hA), B: Hop(B, hB)},
    reactions=reactions,
)

Now we're ready to tun the simulations. We run `T` Monte Carlo steps (default is 1000). How long this takes will depend on the parameter settings above and the hardware this is being run on. With the default parameters on a typical Google Colab node it takes about five minutes. This might be significantly less when run on a modern laptop or desktop machine.

Please note that the goal of the `PyPop` module is not computational efficiency but rather code clarity as it is an educational project. This could easily be made to run 100x faster if proper optimizations are applied, however this would sacrifice the readability of the module's code.

In [ ]:
from tqdm.auto import tqdm

# Initialize list that will hold a view of the lattice state at each step.
# The first entry shows the initial distribution of species site occupants.
numbers = [world.asarrays()]

# Iterate over `T` time steps
# `tqdm` is a tool to display a progress bar.
for _ in tqdm(range(T), smoothing=0, desc="Simulation progress"):
    # Run a single time step
    world.step()
    # Save the state of the lattice (distribution of occupants)
    numbers.append(world.asarrays())

After this has successfully run, we'd like to visualize the results. The code in the cell below displays an interactive graph where you can see the state of the lattice at each step (left side), together with a plot of the species densities over time (right side). 

In [ ]:
# Display matplotlib figures inline (not the interactive notebook extension)
%matplotlib inline
# Load modules
from matplotlib.figure import Figure
from ipywidgets import interact
import numpy as np
from IPython.display import display

# Initialize figure and axes
fig = Figure(figsize=(8, 4), dpi=150)
ax = fig.add_axes([0, 0, 0.5, 1])
# We don't want any decorations on the axes.
ax.axis('off')
plot = fig.add_axes([0.6, 0.1, 0.4, 0.8])

# Extract densities from species distribution arrays
N = size[0]*size[1]
A_densities, B_densities = zip(*[(entry["A"].sum()/N, entry["B"].sum()/N) for entry in numbers])

# Plot the densities
plot.plot(A_densities, color="red", label="A")
plot.plot(B_densities, color="blue", label="B")
# Label the plot
plot.set_yscale("log", base=10)
plot.set_ylabel("Species density [occupants/site]")
plot.set_xlabel("Time [MC steps]")

# This code displays the correct view of the lattice given the MC step
mappable = None
vline = None
def show_image(MC_step=0):
    global mappable, vline  # Hackish, but easy, avoid global if possible.

    # Load array of lattice occupant numbers
    arrays = numbers[MC_step]
    # Create a red and blue image
    # `image` is a WxHx3 image where the three channels at the end correspond to red-green-blue
    image = np.zeros((size[0], size[1], 3), dtype=np.uint8)
    # Make a pixel that has at least one predator red, at least one prey blue and purple if both are present.
    # Leave the green channel alone.
    image[:, :, 0] = 255*(arrays["A"] > 0)
    image[:, :, 2] = 255*(arrays["B"] > 0)
    if mappable is None:
        # We need to create the mappable first, afterwards we can just load data into it.
        mappable = ax.imshow(image)
        # Similar for the vline indicating the MC step we're at.
        vline = plot.axvline(0, color="black")
    else:
        # Load data into mappable and vline.
        mappable.set_data(image)
        vline.set_xdata([MC_step])

    # Finally, display the updated figure.
    display(fig)

# THis lets us have a slider to select the MC step to display.
interact(show_image, MC_step=(0, len(numbers)-1))
None

With the default parameters, what you should see if you scroll through the MC steps above is that initially the preys (`B`) are abundant while the predators (`A`) are sparsely distributed. The `B`s reproduce, however after 20 steps or so, the `A`s start to catch up, consuming `B`s and reproducing. This leads to an initial massive explosion of species `A`, and species `B` is driven almost to extinction. Species `A` runs out of `B` occupants to consume and itself rapidly drops in numbers. This allows `B` to recover, and the cycle repeats a few times. These are the famous Lotka-Volterra oscillations. However, in contrast to the popular mean-field equation model, these oscillations here are damped due to the spatially extended and stochastic nature of this system. After a few hundred steps, the system will have settled into a steady state where species numbers fluctuate around coexistence densities.